### Initialized word vector with pretrain japanese word2vec 

https://github.com/Kyubyong/wordvectors

In [1]:
word_vectors = "-word2vec"

In [2]:
#import cPickle (python2.7)
#http://testpy.hatenablog.com/entry/2017/03/17/000626
import _pickle as cPickle

import numpy as np
from collections import defaultdict, OrderedDict

import re
import sys
import time

In [3]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable

import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# Hyper parameters
num_epochs = 10
num_classes = 3
batch_size = 50
learning_rate = 0.001

### Load pickle data

In [5]:
# the pickle file contains [revs, W, W2, word_idx_map, vocab] # W2 random vectors
x = cPickle.load(open("gensim_tweet.p","rb"), encoding="latin1") # Add encoding="latin1" because got UnicodeDecodeError

### Dataset content
Get data from twitter (reference: http://tech.wonderpla.net/entry/2017/10/10/110000)
- Label 0
    - KEYWORD = "芸能 OR アニメ OR 漫画 OR ドラマ OR ゲーム"            #エンタメ系のキーワード
    - CLASS_LABEL = "\__label__0"

- Label 1
    - KEYWORD = "美容 OR サロン OR エステ OR 化粧 OR 保湿"            #美容系のキーワード
    - CLASS_LABEL = "\__label__1"

- Label 2
    - KEYWORD = "日常 OR 料理 OR 家事 OR 収納 OR 家具"            #暮らし系のキーワード
    - CLASS_LABEL = "\__label__2"

#### See data sample

In [6]:
df = pd.DataFrame(x[0])

# label 0: Entertainment 
df[df['y'] == 0].sample(2)

,num_words,split,text,y
59,33,2,某 違法 サイト が 潰れ て から 数 ヶ月 、 各 漫画 家 や 漫画 原作 者 さん ...,0
82,44,5,ルージュ さん おはよう ござい ます ！ かき ふら い さん の 名義 で これ まで ...,0


In [7]:
# label 1 : Beauty
df[df['y'] == 1].sample(2)

,num_words,split,text,y
1048,72,7,シル キー 2 ウェイ ファンデーション ( spf 20 / pa ++) 透明 感 の ...,1
1069,62,4,ホームページ で は 美容 ・ 健康 ・ ブライダル の お 得 な 情報 を 掲載 し て...,1


In [8]:
# label 2: Life
df[df['y'] == 2].sample(2)

,num_words,split,text,y
1432,52,4,修羅場 家 の 日常 : 結婚式 に 出席 し た 時 、 トイレ に 中座 し て た 友...,2
1742,16,7,日常 生活 に は 支障 ない ん だ けど な 〜 〜 〜 作業 が 辛い,2


In [9]:
revs, W, W2, word_idx_map, vocab = x[0], x[1], x[2], x[3], x[4]

In [10]:
# Get the number of the longest sentence
max_l = np.max(pd.DataFrame(revs)["num_words"])
print("max sentence length: ", max_l)

max sentence length:  100


In [11]:
print('revs',len(x[0])) # number of sentence
print('W', len(x[1])) # W are pretrained word vectors (unknown words are randomly initialized)
print('W2', len(x[2])) # W2 are randomly initialized vectors
print('word_idx_map', len(x[3]))
print('vocab', len(x[4]))

revs 2068
W 10330
W2 10330
word_idx_map 10329
vocab 10329


In [12]:
revs[1]

{'y': 0,
 'text': 'え 、 サラリーマン し ながら 、 商業 漫画 の 仕事 を し て 、 ツイッター に 定期 的 に 漫画 を あげ て 、 さらに コミケ の 原稿 を 作る ！ ？',
 'num_words': 32,
 'split': 1}

In [13]:
if word_vectors=="-rand":
    print("using: random vectors")
    U = W2
elif word_vectors=="-word2vec":
    print("using: word2vec vectors")
    U = W

using: word2vec vectors


In [14]:
U.shape

(10330, 300)

### Make dataset (check original code)
make each sentence an word index map using word_idx_map

In [15]:
def get_idx_from_sent(sent, word_idx_map, max_l=51, k=300, filter_h=5):
    """
    Transforms sentence into a list of indices. Pad with zeroes.
    """
    x = []
    pad = filter_h - 1
    for i in range(pad):
        x.append(0)
    words = sent.split()
    for word in words:
        if word in word_idx_map:
            x.append(word_idx_map[word])
    while len(x) < max_l + 2*pad:
        x.append(0)
    return x

def make_idx_data_cv(revs, word_idx_map, cv, max_l=51, k=300, filter_h=5):
    """
    Transforms sentences into a 2-d matrix.
    """
    train, test = [], []
    for rev in revs:
        sent = get_idx_from_sent(rev["text"], word_idx_map, max_l, k, filter_h) # one sentence
        sent.append(rev["y"])
        if rev["split"]== cv:  # "split" is random number of np.random.randint(0,10)
            test.append(sent)        
        else:  
            train.append(sent)   
    train = np.array(train, dtype="int")
    test = np.array(test, dtype="int")
    return [train, test] 

In [16]:
i = 0
datasets = make_idx_data_cv(revs, word_idx_map, i, max_l, k=300, filter_h=5)

#### Example of sentence to word index map

In [17]:
revs[1]

{'y': 0,
 'text': 'え 、 サラリーマン し ながら 、 商業 漫画 の 仕事 を し て 、 ツイッター に 定期 的 に 漫画 を あげ て 、 さらに コミケ の 原稿 を 作る ！ ？',
 'num_words': 32,
 'split': 1}

In [18]:
print(len(datasets[0][0]))
datasets[0][1] # sentence => word index map padding with zeros

109


array([ 0,  0,  0,  0, 49, 31, 40, 22, 33, 31, 36,  1, 39, 46, 44, 22,  8,
       31, 45, 16, 47, 35, 16,  1, 44, 43,  8, 31, 41, 37, 39, 42, 44, 34,
       38, 48,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0])

In [19]:
print('train data size:', datasets[0].shape)
print('test data size:', datasets[1].shape)

train data size: (1853, 109)
test data size: (215, 109)


### Make dataset (using vectors)

In [20]:
def get_idx_from_sent_2vec(sent, U, word_idx_map, max_l=51, k=300, filter_h=5):
    """
    Transforms sentence into a list of indices. Pad with zeroes.
    """
    pad = filter_h - 1
    x = np.zeros((max_l+2*pad, k))

    words = sent.split()
    # starting after padding
    i = pad
    for word in words:
        if word in word_idx_map:
            x[i] = U[word_idx_map[word]]
            i += 1
    return x

def make_idx_data_cv_2vec(revs, U, word_idx_map, cv, max_l=51, k=300, filter_h=5):
    """
    Transforms sentences into a 2-d matrix.
    """
    train_image, train_label = [], []
    test_image, test_label = [], []
    test_rev = []
    for rev in revs:
        sent = get_idx_from_sent_2vec(rev["text"], U, word_idx_map, max_l, k, filter_h) # one sentence
        if rev["split"] == cv:  # "split" is random number of np.random.randint(0,10)
            test_image.append(sent) 
            test_label.append(rev["y"])
            test_rev.append(rev)
        else:  
            train_image.append(sent)
            train_label.append(rev["y"])
    train_image = np.array(train_image)
    train_label = np.array(train_label)
    test_image = np.array(test_image)
    test_label = np.array(test_label)
    test_rev = np.array(test_rev)
    return (train_image, train_label), (test_image, test_label, test_rev)

In [21]:
t = "effective but too tepid biopic"
t_sent_2vec = get_idx_from_sent_2vec(t, U, word_idx_map, max_l, k=300, filter_h=5)
print("sentence length(before)", len(t_sent_2vec)) # max_l(51)+2*pad(filter_h-1)

sentence length(before) 108


In [22]:
t_sent_2vec[0].shape

(300,)

In [23]:
i = 0
datasets_2vec = make_idx_data_cv_2vec(revs, U, word_idx_map, i, max_l, k=300, filter_h=5)

In [24]:
(train_image, train_label), (test_image, test_label, test_rev) = datasets_2vec

#### Example of sentence to vector

In [25]:
revs[1]

{'y': 0,
 'text': 'え 、 サラリーマン し ながら 、 商業 漫画 の 仕事 を し て 、 ツイッター に 定期 的 に 漫画 を あげ て 、 さらに コミケ の 原稿 を 作る ！ ？',
 'num_words': 32,
 'split': 1}

In [26]:
p2 = pd.DataFrame(train_image[1])
print(p2.shape)
p2.head(10)

(108, 300)


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.047938,-0.012884,-0.056857,-0.012662,-0.017988,0.017350,0.047852,-0.029005,-0.001137,0.026639,...,-0.045853,0.016910,-0.039848,-0.064570,-0.058615,0.009916,-0.014220,0.107975,0.116522,0.110864
5,-0.071014,-0.010093,-0.048892,0.049935,0.004244,0.059785,0.005612,-0.127936,-0.039739,0.004968,...,0.103655,-0.051866,0.087268,0.056497,0.000689,-0.005126,0.130839,-0.009686,0.003536,0.077469
6,-0.110441,-0.025591,-0.066245,0.078289,-0.043802,-0.002915,-0.022811,-0.006989,-0.010652,0.029903,...,-0.025297,-0.042852,-0.079471,-0.085797,-0.026410,0.000728,-0.072049,-0.070150,0.079390,-0.112369
7,0.048331,0.033419,-0.031500,-0.071572,0.066776,-0.022589,-0.016808,0.019989,-0.108131,-0.035041,...,-0.081411,0.009484,-0.001005,0.088263,-0.018893,0.041408,-0.079723,-0.078454,-0.043328,-0.036232
8,-0.128078,0.067364,0.004988,0.013444,-0.015906,-0.113039,-0.053078,0.010158,-0.043217,-0.024071,...,-0.120665,0.040173,0.007844,-0.046066,-0.053584,0.060227,-0.129146,-0.029178,0.015250,0.054799
9,-0.071014,-0.010093,-0.048892,0.049935,0.004244,0.059785,0.005612,-0.127936,-0.039739,0.004968,...,0.103655,-0.051866,0.087268,0.056497,0.000689,-0.005126,0.130839,-0.009686,0.003536,0.077469


### Make dataset iterable

In [27]:
t_image = torch.FloatTensor(train_image).reshape(-1, 1, max_l + 2 * (5-1), 300)
t_label = torch.LongTensor(train_label)
train_dataset = list(zip(t_image, t_label))
len(train_dataset)

1853

In [28]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)


In [29]:
for images, labels in train_loader:
    print('images:', images.shape, '\nlabels:', labels.shape)
    print(images[1][0])
    print(labels[1])
    break

images: torch.Size([50, 1, 108, 300]) 
labels: torch.Size([50])
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor(0)


In [30]:
c_image = torch.FloatTensor(test_image).reshape(-1, 1, max_l + 2 * (5-1), 300)
c_label = torch.LongTensor(test_label)
test_dataset = list(zip(c_image, c_label, test_rev))
len(test_dataset)

215

In [31]:
# Data loader
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size, 
                                           shuffle=False)


### Model parameters (original code)

In [32]:
filter_hs=[3, 4, 5]
hidden_units=[100, num_classes]
batch_size=50
img_w=300
img_h = len(datasets[0][0])-1  # sentence length (subtracted 1 for y label)

filter_w = img_w    
feature_maps = hidden_units[0]
filter_shapes = []
pool_sizes = []
for filter_h in filter_hs:
    filter_shapes.append((feature_maps, 1, filter_h, filter_w))
    pool_sizes.append((img_h-filter_h+1, img_w-filter_w+1))


In [33]:
print(filter_shapes)
image_shape = (batch_size, 1, img_h, img_w)
print('one batch train', image_shape)
print(pool_sizes)

[(100, 1, 3, 300), (100, 1, 4, 300), (100, 1, 5, 300)]
one batch train (50, 1, 108, 300)
[(106, 1), (105, 1), (104, 1)]


### Padding & Stride
- Output size

    $ O = \frac {W-K+2P}{S} + 1 $
    - O: output h/w
    - W: input h/w
    - K: filter size(kernel size)
    - P: padding
        - $  P = \frac {K-1}{2} $
    - S: stride

### Network

- Theano:
    - conv_layer: LeNetConvPoolLayer
    - classifier: MLPDropout

### Model:


```
Network
Input ->
Conv -> ReLU -> MaxPool |
Conv -> ReLU -> MaxPool | -> concat
Conv -> ReLU -> MaxPool |
Fully Connected Layer(Logits -> Softmax) -> Labels
```

```
Convolutional layer formula:
- Filter 1(Kernel) size K = 3 => (3 x 300)
- P(same padding) P = (3-1)/2=1
- S(stride) S = 1
- in_channels = 1
- out_channels (int) – Number of channels produced by the convolution = 100
Pooling layer formula:
- K
```

```
*Filter dimensions*:
Conv1 (W_conv, (100, 1, 3, 300))
Conv1 (b_conv, (100,))
Conv2 (W_conv, (100, 1, 4, 300))
Conv2 (b_conv, (100,))
Conv3 (W_conv, (100, 1, 5, 300))
Conv3 (b_conv, (100,))

*Layer input dimensions*:
- Input image(64, 300) 

----------------------------------------------------------------------
|  Conv1  (100, 3, 300)   Conv2  (100, 4, 300)   Conv3  (100, 5, 300) |
|  MaxPool (100, 62, 1)   MaxPool (100, 61, 1)   MaxPool (100, 60, 1) |
-----------------------------Concat ----------------------------------

- Concatenated (100, 1, 1) + (100, 1, 1) + (100, 1, 1) => (300, 1, 1) 

- Fully Connected Layer(Logits (100, 1) -> Logits (2, 1) -> Softmax) -> Labels
```

In [34]:
class ConvPoolLayer(nn.Module):
    def __init__(self, num_classes=2):
        super(ConvPoolLayer, self).__init__()

        # Layer 1: conv - relu - conv- relu - pool
        self.ngram1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=feature_maps, kernel_size=(filter_hs[0], img_w), stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=pool_sizes[0], stride=None))
        self.ngram2 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=feature_maps, kernel_size=(filter_hs[1], img_w), stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=pool_sizes[1], stride=None))
        self.ngram3 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=feature_maps, kernel_size=(filter_hs[2], img_w), stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=pool_sizes[2], stride=None))
        
        # Fully Connected 1 (readout)
        self.fc = nn.Sequential(
            nn.Linear(feature_maps * 3, hidden_units[0]),
            nn.ReLU(),
            nn.Linear(hidden_units[0], num_classes))

        # Initialize all parameters using kaiming normalization
        self.init_weights_kaiming()
    
    def init_weights_kaiming(self):
        #Use kaiming normalization to initialize the parameters
        for layer in [self.ngram1, self.ngram2, self.ngram3, self.fc]:
            for m in layer:
                if type(m) == nn.Conv2d or type(m) == nn.Linear:
                    m.weight = nn.init.kaiming_normal_(m.weight)

    def forward(self, x):

        out1 = self.ngram1(x)
        out2 = self.ngram2(x)
        out3 = self.ngram3(x)
        out = torch.cat((out1, out2, out3), 1)
        
        out = out.view(out.size(0), -1)
    
        # Linear function (readout)
        out = self.fc(out)
        
        return out

In [35]:
model = ConvPoolLayer(num_classes)
print(model)

ConvPoolLayer(
  (ngram1): Sequential(
    (0): Conv2d(1, 100, kernel_size=(3, 300), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(106, 1), stride=(106, 1), padding=0, dilation=1, ceil_mode=False)
  )
  (ngram2): Sequential(
    (0): Conv2d(1, 100, kernel_size=(4, 300), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(105, 1), stride=(105, 1), padding=0, dilation=1, ceil_mode=False)
  )
  (ngram3): Sequential(
    (0): Conv2d(1, 100, kernel_size=(5, 300), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(104, 1), stride=(104, 1), padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=300, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=3, bias=True)
  )
)


In [36]:
criterion = nn.CrossEntropyLoss()

In [37]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [38]:
for p in model.parameters():
    print(p.shape)

torch.Size([100, 1, 3, 300])
torch.Size([100])
torch.Size([100, 1, 4, 300])
torch.Size([100])
torch.Size([100, 1, 5, 300])
torch.Size([100])
torch.Size([100, 300])
torch.Size([100])
torch.Size([3, 100])
torch.Size([3])


### Train model

In [ ]:
avg_losses = list()
iter = 0

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        total_loss = list()
        
        # Load images as Variable
        images = Variable(images) # Now we dont need to resize like images.view(xx)
        labels = Variable(labels)
        
        # Clear gradients w.r.t parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        outputs = model(images)
        
        # Calculate Loss: Softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        
        # Getting gradients w.r.t paramters
        loss.backward()
        
        # Track loss to plot the result
        total_loss.append(loss.item())
        
        # Updating parameters
        optimizer.step()
        
        iter += 1
        
        if iter % 50 == 0:
            # Print Loss
            print('Iteration: {}. Loss: {}.'.format(iter, loss.item()))
            avg_loss = np.divide(np.sum(total_loss), len(total_loss))
            avg_losses.append(avg_loss)

Iteration: 50. Loss: 0.16025562584400177.
Iteration: 100. Loss: 0.028041373938322067.
Iteration: 150. Loss: 0.010904279537498951.
Iteration: 200. Loss: 0.0031761773861944675.
Iteration: 250. Loss: 0.05298465117812157.


### Plot the loss vs iteration

In [ ]:
x_axis = np.arange(len(avg_losses))
plt.plot(x_axis, avg_losses, label='train')
plt.xlabel('iterations (x100)')
plt.ylabel('loss')
plt.show()

### Test model with test dataset

In [ ]:
n_test = len(test_loader) * batch_size
wrong_predictions = []

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels, revs in test_loader:
        images = Variable(images)
        labels = Variable(labels)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # See which are error predictions
        result = (predicted == labels)
        err_imgs = images[result == 0] # 0 means wrong prediction
        err_labels = labels[result == 0]
        err_p = outputs[result == 0]
        err_outputs = predicted[result == 0]
        err_texts = np.array(revs['text'])[np.array((result == 0).numpy(), dtype=np.bool)]
        for img, lbl, p, out, text in zip(err_imgs, err_labels, err_p, err_outputs, err_texts):
            wrong_predictions.append((img, lbl, p, out, text))
     
    print('Test Accuracy of the model on the {} test images: {} %'.format(n_test, 100 * correct / total))

In [ ]:
label_names = ['エンタメ系', '美容系', '暮らし系']
# unpack img, lbl, out, text
for img, lbl, p, out, text in wrong_predictions:
    print('True label: ', lbl.item(), '\t', label_names[lbl.item()])
    print('Predition: ', out.item(), '\t', label_names[out.item()])
    print('Possibility:', p)
    print(text, '\n')
    

### Test model with train dataset

In [ ]:
n_train = len(train_loader) * batch_size

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in train_loader:
        images = Variable(images)
        labels = Variable(labels)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print('Test Accuracy of the model on the {} train images: {} %'.format(n_train, 100 * correct / total))

### Predict single inputted text

#### Input the text to predict (change the text below)

In [ ]:
# text_to_predict = "お風呂掃除でいつも落ちなかった溝にある黒カビが家事えもんの塩素系漂白剤＋片栗粉でほぼ真っ白になって感動 家事えもんのテクニック凄い～！"

text_to_predict = "コスメの最安値が見つけられるアプリ💄💋メイク動画とか 美容情報も載ってるし最高😆🙌📲http://goo.gl/K5Fmea 女子にはほんとに助かる〜💗"

In [ ]:
import sys, os
sys.path.append(os.pardir)
from process_data import build_single_data_cv

In [ ]:
def single_make_idx_data_cv_2vec(revs, U, word_idx_map, max_l=51, k=300, filter_h=5):
    """
    Transforms sentences into a 2-d matrix.
    """
    test_image, test_label = [], []
    test_rev = []
    for rev in revs:
        sent = get_idx_from_sent_2vec(rev["text"], U, word_idx_map, max_l, k, filter_h) # one sentence
        test_image.append(sent) 
        test_label.append(rev["y"])
        test_rev.append(rev)

    test_image = np.array(test_image)
    test_label = np.array(test_label)
    test_rev = np.array(test_rev)
    return test_image, test_label, test_rev

In [ ]:
single_revs, _ = build_single_data_cv(text_to_predict)
single_data_2vec = single_make_idx_data_cv_2vec(single_revs, U, word_idx_map, max_l, k=300, filter_h=5)

In [ ]:
model.eval()
with torch.no_grad():
    images, labels, revs = single_data_2vec
    images = Variable(torch.Tensor(images.reshape(1, 1, -1, 300)))
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    print('Prediction:', predicted.item(), label_names[predicted.item()])
    print('Text:', revs[0]['text'])